In [29]:
import numpy as np
import pandas as pd
import sys
import os
import csv
import ipdb
import pickle
from collections import OrderedDict, defaultdict
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [30]:
CONFIG = {
    'week': 'week12',
    'current_week': 12
    # 'calling_list': sys.argv[2],
    # 'round_robin_list': sys.argv[3]
}

In [31]:
np.random.seed(1)

df = pd.read_csv('outputs/analysis_lists/all_analysis_week_12.csv')
df = df.sort_values('{}_whittle'.format(CONFIG['week']), ascending=False)
df

,Unnamed: 0,user_id,cluster,arm,registration_date,intervention_week,week1_whittle,after_intervention_E/C,week12_whittle,week0_E/C,...,week2_E/C,week3_E/C,week4_E/C,week5_E/C,week6_E/C,week7_E/C,week8_E/C,week9_E/C,week10_E/C,week11_E/C
22393,22467,2403430,29,control,1159.0,NaN,-0.387939,0/0,0.865692,1/2,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0
100,21,2392087,29,control,1142.0,NaN,0.865692,0/0,0.865692,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/2,0/1
90,29,2398871,29,rmab,1150.0,week6,0.865692,1/9,0.865692,0/2,...,0/2,0/2,0/2,0/2,0/0,0/1,1/2,0/2,0/2,0/2
91,30,2394628,29,round_robin,1151.0,NaN,0.865692,0/0,0.865692,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0
94,33,2388450,29,control,1144.0,NaN,0.865692,0/0,0.865692,0/2,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8618,8635,2397443,34,control,1151.0,NaN,0.145004,0/0,-0.882812,1/2,...,1/2,0/0,1/2,1/2,1/2,0/2,1/2,0/1,0/1,0/1
8631,8629,2408039,34,control,1164.0,NaN,0.145004,0/0,-0.882812,2/2,...,2/2,2/2,2/2,2/2,2/2,2/2,2/2,0/2,0/2,0/0
8636,8618,2406148,34,control,1157.0,NaN,0.145004,0/0,-0.882812,2/2,...,1/1,0/0,0/0,0/0,1/1,0/0,0/0,0/0,0/0,0/0
8639,8614,2408765,34,rmab,1160.0,NaN,0.145004,0/0,-0.882812,2/2,...,1/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [32]:
df2 = pd.read_csv('outputs/individual_clustering/weekly_kmeans_pilot_stats_40.csv')

rmab_list = pd.read_csv('outputs/pilot_outputs/rmab_pilot.csv')['user_id'].to_list()
round_robin_list = pd.read_csv('outputs/pilot_outputs/round_robin_pilot.csv')['user_id'].to_list()
control_list = pd.read_csv('outputs/pilot_outputs/control_pilot.csv')['user_id'].to_list()

rmab_group = df[df['user_id'].isin(rmab_list)]
rmab_group = rmab_group.sort_values('{}_whittle'.format(CONFIG['week']), ascending=False)

round_robin_group = df2[df2['user_id'].isin(round_robin_list)]
round_robin_group = round_robin_group.sort_values('registration_date', ascending=True)

rmab_user_ids = rmab_group['user_id'].to_list()
round_robin_user_ids = round_robin_group['user_id'].to_list()

In [33]:
intervention_benefit = {'rmab': [], 'round_robin': [], 'control': []}

all_user_ids = set(df['user_id'].to_list())

full_mat = {'rmab': [], 'round_robin': [], 'control': []}

with open('policy_dump.pkl', 'rb') as fr:
    pilot_user_ids, pilot_static_features, cls, cluster_transition_probabilities, m_values, q_values = pickle.load(fr)
fr.close()

arm_to_cat = {'rmab': [1,0], 'round_robin': [0,1], 'control': [0,0]}
X_mat, Y_mat = [], []

/Users/vermashresth/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/vermashresth/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [34]:
mask = {'rmab':[0]*len(pilot_user_ids), 'round_robin':[0]*len(pilot_user_ids), 'control':[0]*len(pilot_user_ids)}
ctr = 0
for idx, user_id in tqdm(enumerate(pilot_user_ids)):
    if user_id not in all_user_ids:
        continue

    curr_mat = []
    curr_row = df[df['user_id'] == user_id]

    # check_row = df_check[df_check['user_id'] == user_id]

    arm = curr_row['arm'].item()
    count = 0

    week0e = int(curr_row['week{}_E/C'.format(0)].item().split('/')[0])

    if week0e:
        curr_mat.append(1)
    else:
        curr_mat.append(0)

    for i in range(1,12):
        nume = int(curr_row['week{}_E/C'.format(i)].item().split('/')[0])
        if nume > 0:
            count += 1
            curr_mat.append(1)
        else:
            curr_mat.append(0)
    
    X_mat.append(list(pilot_static_features[idx, :]) + [curr_mat[0]] + arm_to_cat[arm])
    Y_mat.append(count)
    mask[arm][ctr] = 1
    ctr+=1

    full_mat[arm].append(np.array(curr_mat, dtype=np.int))
mask = {i:np.array(mask[i][:ctr]) for i in mask}
X_mat = np.array(X_mat)
Y_mat = np.array(Y_mat)

13365it [00:13, 1021.53it/s]


KeyboardInterrupt: 

## Multi Timestep

In [35]:
X_mat, Y_mat = [], [[] for i in range(1,12)]

mask = {'rmab':[0]*len(pilot_user_ids), 'round_robin':[0]*len(pilot_user_ids), 'control':[0]*len(pilot_user_ids)}
ctr = 0
for idx, user_id in tqdm(enumerate(pilot_user_ids)):
    if user_id not in all_user_ids:
        continue

    curr_mat = []
    curr_row = df[df['user_id'] == user_id]

    # check_row = df_check[df_check['user_id'] == user_id]

    arm = curr_row['arm'].item()
    count = 0

    week0e = int(curr_row['week{}_E/C'.format(0)].item().split('/')[0])

    if week0e:
        curr_mat.append(1)
    else:
        curr_mat.append(0)

    for i in range(1,12):
        nume = int(curr_row['week{}_E/C'.format(i)].item().split('/')[0])
        if nume > 0:
            count += 1
            curr_mat.append(1)
        else:
            curr_mat.append(0)
        Y_mat[i-1].append(count)
    X_mat.append(list(pilot_static_features[idx, :]) + [curr_mat[0]] + arm_to_cat[arm])
    mask[arm][ctr] = 1
    ctr+=1

    full_mat[arm].append(np.array(curr_mat, dtype=np.int))


23008it [00:23, 986.29it/s] 


In [36]:
columns = [
    "enroll_gest_age",
    "enroll_delivery_status",
    "g",
    "p",
    "s",
    "l",
    "a",
    "days_to_first_call",
    "age_0",
    "age_1",
    "age_2",
    "age_3",
    "age_4",
    "language_2",
    "language_3",
    "language_4",
    "language_5",
    "education_1",
    "education_2",
    "education_3",
    "education_4",
    "education_5",
    "education_6",
    "education_7",
    "phone_owner_0",
    "phone_owner_1",
    "phone_owner_2",
    "call_slots_1",
    "call_slots_2",
    "call_slots_3",
    "call_slots_4",
    "call_slots_5",
    "call_slots_6",
    "ChannelType_0",
    "ChannelType_1",
    "ChannelType_2",
    "income_bracket_-1",
    "income_bracket_0",
    "income_bracket_1",
    "income_bracket_2",
    "income_bracket_3",
    "income_bracket_4",
    "income_bracket_5",
    "income_bracket_6",
    "start_state",
    "exp_arm_rmab",
    "exp_arm_rr",
]

In [40]:
print('With feature reduction')
mask = {i:np.array(mask[i][:ctr]) for i in mask}
X_mat = np.array(X_mat)
Y_mat = np.array(Y_mat)

income_cols = [i for i in columns if i.startswith('income')]
education_cols = [i for i in columns if i.startswith('education')]
language_cols = [i for i in columns if i.startswith('language')]
slots_cols = [i for i in columns if i.startswith('call_slots')]

for t in range(1,12):
    drop_cols = income_cols
    
    print(f'Significance for T = {t}')
    print('\tRMAB vs RR')
    Y_mat_df = pd.DataFrame(Y_mat[t-1][mask['control']==0], columns = ['output'])
    X_mat_df = pd.DataFrame(X_mat[mask['control']==0][:, :-1], columns = columns[:-1])

    mod = sm.OLS(Y_mat_df,X_mat_df.drop(columns=drop_cols))
    fii = mod.fit()
    var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[-1]
    print(f'\tCoef: {coef}, p_val: {p_val}')
    print('\n')
    print('\tRMAB vs Control')
    Y_mat_df = pd.DataFrame(Y_mat[t-1][mask['round_robin']==0], columns = ['output'])
    X_mat_df = pd.DataFrame(X_mat[mask['round_robin']==0][:, :-1], columns = columns[:-1])

    mod = sm.OLS(Y_mat_df,X_mat_df.drop(columns=drop_cols))
    fii = mod.fit()
    var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[-1]
    print(f'\tCoef: {coef}, p_val: {p_val}')
    print('\n')
    print('\tRR vs Control')
    Y_mat_df = pd.DataFrame(Y_mat[t-1][mask['rmab']==0], columns = ['output'])
    X_mat_df = pd.DataFrame(np.concatenate([X_mat[mask['rmab']==0][:, :-2],
                                            X_mat[mask['rmab']==0][:, -1:]], axis=1),
                            columns = columns[:-2]+columns[-1:])

    mod = sm.OLS(Y_mat_df,X_mat_df.drop(columns=drop_cols))
    fii = mod.fit()
    var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[-1]
    print(f'\tCoef: {coef}, p_val: {p_val}')
    print('\n\n')

With feature reduction
Significance for T = 1
	RMAB vs RR
	Coef:     0.0081, p_val:  0.195


	RMAB vs Control
	Coef:     0.0059, p_val:  0.346


	RR vs Control
	Coef:    -0.0023, p_val:  0.720



Significance for T = 2
	RMAB vs RR
	Coef:     0.0203, p_val:  0.058


	RMAB vs Control
	Coef:     0.0172, p_val:  0.106


	RR vs Control
	Coef:    -0.0033, p_val:  0.757



Significance for T = 3
	RMAB vs RR
	Coef:     0.0238, p_val:  0.114


	RMAB vs Control
	Coef:     0.0157, p_val:  0.292


	RR vs Control
	Coef:    -0.0087, p_val:  0.561



Significance for T = 4
	RMAB vs RR
	Coef:     0.0284, p_val:  0.140


	RMAB vs Control
	Coef:     0.0135, p_val:  0.482


	RR vs Control
	Coef:    -0.0162, p_val:  0.401



Significance for T = 5
	RMAB vs RR
	Coef:     0.0300, p_val:  0.204


	RMAB vs Control
	Coef:     0.0233, p_val:  0.317


	RR vs Control
	Coef:    -0.0077, p_val:  0.743



Significance for T = 6
	RMAB vs RR
	Coef:     0.0269, p_val:  0.335


	RMAB vs Control
	Coef:     0.0244, p_val:

In [38]:
print('No feature reduction')
mask = {i:np.array(mask[i][:ctr]) for i in mask}
X_mat = np.array(X_mat)
Y_mat = np.array(Y_mat)

for t in range(1,12):
    print(f'Significance for T = {t}')
    print('\tRMAB vs RR')
    Y_mat_df = pd.DataFrame(Y_mat[t-1][mask['control']==0], columns = ['output'])
    X_mat_df = pd.DataFrame(X_mat[mask['control']==0][:, :-1], columns = columns[:-1])

    mod = sm.OLS(Y_mat_df,X_mat_df)
    fii = mod.fit()
    var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[-1]
    print(f'\tCoef: {coef}, p_val: {p_val}')
    print('\n')
    print('\tRMAB vs Control')
    Y_mat_df = pd.DataFrame(Y_mat[t-1][mask['round_robin']==0], columns = ['output'])
    X_mat_df = pd.DataFrame(X_mat[mask['round_robin']==0][:, :-1], columns = columns[:-1])

    mod = sm.OLS(Y_mat_df,X_mat_df)
    fii = mod.fit()
    var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[-1]
    print(f'\tCoef: {coef}, p_val: {p_val}')
    print('\n')
    print('\tRR vs Control')
    Y_mat_df = pd.DataFrame(Y_mat[t-1][mask['rmab']==0], columns = ['output'])
    X_mat_df = pd.DataFrame(np.concatenate([X_mat[mask['rmab']==0][:, :-2],
                                            X_mat[mask['rmab']==0][:, -1:]], axis=1),
                            columns = columns[:-2]+columns[-1:])

    mod = sm.OLS(Y_mat_df,X_mat_df)
    fii = mod.fit()
    var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[-1]
    print(f'\tCoef: {coef}, p_val: {p_val}')
    print('\n\n')

No feature reduction
Significance for T = 1
	RMAB vs RR
	Coef:     0.0079, p_val:  0.209


	RMAB vs Control
	Coef:     0.0056, p_val:  0.371


	RR vs Control
	Coef:    -0.0023, p_val:  0.717



Significance for T = 2
	RMAB vs RR
	Coef:     0.0197, p_val:  0.066


	RMAB vs Control
	Coef:     0.0167, p_val:  0.118


	RR vs Control
	Coef:    -0.0034, p_val:  0.753



Significance for T = 3
	RMAB vs RR
	Coef:     0.0228, p_val:  0.129


	RMAB vs Control
	Coef:     0.0148, p_val:  0.320


	RR vs Control
	Coef:    -0.0087, p_val:  0.561



Significance for T = 4
	RMAB vs RR
	Coef:     0.0270, p_val:  0.162


	RMAB vs Control
	Coef:     0.0122, p_val:  0.522


	RR vs Control
	Coef:    -0.0161, p_val:  0.403



Significance for T = 5
	RMAB vs RR
	Coef:     0.0283, p_val:  0.229


	RMAB vs Control
	Coef:     0.0219, p_val:  0.348


	RR vs Control
	Coef:    -0.0077, p_val:  0.741



Significance for T = 6
	RMAB vs RR
	Coef:     0.0252, p_val:  0.367


	RMAB vs Control
	Coef:     0.0228, p_val:  

In [20]:
pd.Series(Y_mat[mask['control']==0]).value_counts()

0     4188
11    2477
1     1389
10    1244
2      931
3      851
9      831
4      770
8      753
7      654
6      629
5      619
dtype: int64

In [23]:
print('RMAB vs RR')
Y_mat_df = pd.DataFrame(Y_mat[mask['control']==0], columns = ['output'])
X_mat_df = pd.DataFrame(X_mat[mask['control']==0][:, :-1], columns = columns[:-1])

mod = sm.OLS(Y_mat_df,X_mat_df)
fii = mod.fit()
fii.summary()


RMAB vs RR


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 output   R-squared:                       0.482
Model:                            OLS   Adj. R-squared:                  0.480
Method:                 Least Squares   F-statistic:                     394.9
Date:                Tue, 17 Aug 2021   Prob (F-statistic):               0.00
Time:                        23:17:01   Log-Likelihood:                -39061.
No. Observations:               15336   AIC:                         7.820e+04
Df Residuals:                   15299   BIC:                         7.848e+04
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
enroll_gest_age           -0.0227      0.003     -7.862      0.000      -0.028      -0.017
enroll_delivery_status    -0.6880      0.089     -7.746      0.000      -0.862      -0.514
g                          0.0541      0.061      0.889      0.374      -0.065       0.174
p                          0.0066      0.127      0.052      0.959      -0.242       0.255
s                         -0.2440      0.213     -1.148      0.251      -0.661       0.173
l                         -0.1087      0.122     -0.893      0.372      -0.348       0.130
a                         -0.1827      0.136     -1.341      0.180      -0.450       0.084
days_to_first_call         0.0027      0.003      0.879      0.379      -0.003       0.009
age_0                      0.1577      0.076      2.064      0.039       0.008       0.307
age_1                      0.0313      0.058      0.537      0.592      -0.083       0.146
age_2                      0.2000      0.063      3.168      0.002       0.076       0.324
age_3                      0.2878      0.094      3.048      0.002       0.103       0.473
age_4                      0.5238      0.188      2.792      0.005       0.156       0.891
language_2                 0.3888      0.095      4.078      0.000       0.202       0.576
language_3                 0.5024      0.097      5.199      0.000       0.313       0.692
language_4              5.981e-17   1.09e-16      0.547      0.584   -1.54e-16    2.74e-16
language_5                 0.3093      0.235      1.317      0.188      -0.151       0.770
education_1                0.0952      0.105      0.905      0.366      -0.111       0.301
education_2               -0.0498      0.063     -0.795      0.427      -0.173       0.073
education_3                0.0295      0.054      0.546      0.585      -0.076       0.135
education_4                0.1140      0.058      1.965      0.049       0.000       0.228
education_5                0.4329      0.074      5.844      0.000       0.288       0.578
education_6                0.5254      0.154      3.414      0.001       0.224       0.827
education_7                0.0535      0.095      0.564      0.572      -0.132       0.239
phone_owner_0              0.6227      0.074      8.436      0.000       0.478       0.767
phone_owner_1              0.5507      0.090      6.142      0.000       0.375       0.727
phone_owner_2              0.0271      0.164      0.165      0.869      -0.294       0.348
call_slots_1              -0.0523      0.080     -0.650      0.516      -0.210       0.105
call_slots_2               0.2310      0.082      2.828      0.005       0.071       0.391
call_slots_3               0.1846      0.050      3.714      0.000       0.087       0.282
call_slots_4               0.1650      0.065      2.541      0.011       0.038  

In [33]:
reg = LinearRegression().fit(X_mat, Y_mat)

fimp = reg.coef_
fimp = fimp.reshape((fimp.shape[0], 1))
fimp_idx = np.concatenate((fimp, np.arange(fimp.shape[0]).reshape((fimp.shape[0], 1))), axis=1)

Y_mat_df = pd.DataFrame(Y_mat, columns = ['output'])
X_mat_df = pd.DataFrame(X_mat, columns = columns)

mod = sm.OLS(Y_mat_df,X_mat_df)
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']
p_values = p_values.to_list()
sorted_arr = fimp_idx[(-fimp_idx)[:, 0].argsort()]

In [219]:
fii.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 output   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                     422.7
Date:                Tue, 17 Aug 2021   Prob (F-statistic):               0.00
Time:                        17:22:53   Log-Likelihood:                -27506.
No. Observations:               15336   AIC:                         5.509e+04
Df Residuals:                   15299   BIC:                         5.537e+04
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
enroll_gest_age           -0.0075      0.001     -5.493      0.000      -0.010      -0.005
enroll_delivery_status    -0.2888      0.042     -6.907      0.000      -0.371      -0.207
g                          0.0214      0.029      0.747      0.455      -0.035       0.078
p                         -0.0126      0.060     -0.211      0.833      -0.130       0.104
s                         -0.1247      0.100     -1.246      0.213      -0.321       0.071
l                         -0.0250      0.057     -0.436      0.662      -0.137       0.087
a                         -0.0706      0.064     -1.102      0.271      -0.196       0.055
days_to_first_call         0.0007      0.001      0.455      0.649      -0.002       0.003
age_0                      0.0925      0.036      2.572      0.010       0.022       0.163
age_1                      0.0175      0.027      0.638      0.524      -0.036       0.071
age_2                      0.0738      0.030      2.484      0.013       0.016       0.132
age_3                      0.1092      0.044      2.458      0.014       0.022       0.196
age_4                      0.2206      0.088      2.498      0.013       0.047       0.394
language_2                 0.1482      0.045      3.302      0.001       0.060       0.236
language_3                 0.1780      0.045      3.913      0.000       0.089       0.267
language_4              6.995e-18   5.15e-17      0.136      0.892   -9.39e-17    1.08e-16
language_5                 0.1875      0.111      1.695      0.090      -0.029       0.404
education_1                0.0523      0.050      1.056      0.291      -0.045       0.149
education_2               -0.0064      0.029     -0.216      0.829      -0.064       0.051
education_3                0.0130      0.025      0.512      0.608      -0.037       0.063
education_4                0.0546      0.027      1.998      0.046       0.001       0.108
education_5                0.1956      0.035      5.610      0.000       0.127       0.264
education_6                0.2407      0.072      3.323      0.001       0.099       0.383
education_7               -0.0362      0.045     -0.811      0.417      -0.124       0.051
phone_owner_0              0.2652      0.035      7.633      0.000       0.197       0.333
phone_owner_1              0.2173      0.042      5.147      0.000       0.135       0.300
phone_owner_2              0.0312      0.077      0.404      0.686      -0.120       0.182
call_slots_1              -0.0267      0.038     -0.705      0.481      -0.101       0.048
call_slots_2               0.0801      0.038      2.081      0.037       0.005       0.155
call_slots_3               0.1026      0.023      4.386      0.000       0.057       0.148
call_slots_4               0.0552      0.031      1.804      0.071      -0.005  

In [216]:
class_scores = defaultdict(lambda: 0)

for i in range(fimp_idx.shape[0]):
    col_name = columns[int(sorted_arr[i, 1])]
    curr_score = round(sorted_arr[i, 0], 3)
    print('{}\t{}\t{}'.format(col_name, curr_score, p_values[i]))
    if 'age_' in col_name:
        class_scores['age'] += curr_score
    if 'education_' in col_name:
        class_scores['education'] += curr_score
    if 'income_bracket_' in col_name:
        class_scores['income_bracket'] += curr_score
    if 'language_' in col_name:
        class_scores['language'] += curr_score
    if 'exp_arm_' in col_name:
        class_scores['experimental_arm'] += curr_score

print('\n\n')
for k in class_scores:
    print(k, class_scores[k])

start_state	2.777	0.0
education_6	0.167	0.0008936569608681438
income_bracket_2	0.16	1.3242264699329557e-10
ChannelType_1	0.132	2.507456779535083e-48
education_5	0.122	2.0535255235705195e-08
age_4	0.118	0.01250306266586105
phone_owner_0	0.094	2.429555092514537e-14
call_slots_5	0.09	1.2063247740052445e-07
income_bracket_3	0.084	0.00047644167089510355
phone_owner_1	0.046	2.686572948174516e-07
income_bracket_1	0.046	0.0007946016600647585
call_slots_6	0.042	6.051468363057215e-06
exp_arm_rmab	0.028	0.22896936073026117
income_bracket_4	0.022	0.12012333564062473
g	0.021	0.45490926982190616
call_slots_3	0.017	1.163562794253482e-05
language_5	0.016	0.09006411536764253
income_bracket_6	0.008	0.5060425264845194
language_3	0.007	9.159898572016126e-05
age_3	0.007	0.013997175323903611
days_to_first_call	0.001	0.6493652972553704
income_bracket_-1	0.0	0.20221672694490764
ChannelType_2	0.0	0.6394873014994227
language_4	-0.0	0.8918506605122378
call_slots_2	-0.006	0.037413139053456286
enroll_gest_age	-0.0

,user_id,phone_no,lmp,enroll_gest_age,project_id,manager_id,call_slots,enroll_delivery_status,delivery,language,...,phone_owner_husband,phone_owner_neighbor,phone_owner_woman,income_bracket_0-5000,income_bracket_10000-15000,income_bracket_15000-20000,income_bracket_20000-25000,income_bracket_25000-30000,income_bracket_30000 and above,income_bracket_5000-10000
0,2385610,934332f7f2dac0b09edc5572fcbcded8,2020-11-14,13,3,0,3,0,0,2,...,0,0,1,0,0,0,0,0,0,1
1,2385611,d154839aef700aeadd4aeea972ec739b,2020-11-05,14,3,0,3,0,0,2,...,0,0,1,0,0,0,0,0,0,1
2,2385612,1a1893861b32ea57cd86f2c268329995,2020-12-20,8,3,0,6,0,0,3,...,0,0,1,0,0,0,0,0,0,1
3,2385613,608ff805950471b63b183054b60bf3df,2020-11-25,11,3,0,3,0,0,3,...,0,0,1,0,0,0,0,0,0,1
4,2385614,d1fd584498c799e3fbda7d9e38f543bd,2020-07-09,31,3,0,3,0,0,2,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23522,2409132,9eb2c55138cc3198edef39e557e51a09,2021-01-14,8,3,0,1,0,0,3,...,0,0,1,0,0,0,0,0,0,1
23523,2409133,70ae26e929a8625fc32db522954814be,2020-06-12,39,3,0,3,0,1,2,...,0,0,1,0,1,0,0,0,0,0
23524,2409134,4b16fb66d8899412f57328a720cba0a6,2020-09-11,26,3,0,3,0,0,2,...,0,0,1,0,0,0,0,0,0,1
23525,2409135,ee32899ab102835a9a47ca2f43753089,2020-10-16,21,3,0,4,0,0,2,...,0,0,1,0,0,0,0,0,0,1


In [127]:
pilot_pd_data.shape

(23527, 67)

In [27]:
import scipy.stats as stats
import pandas as pd


pilot_pd_data = pd.read_csv("feb16-mar15_data/beneficiary/ai_registration-20210216-20210315.csv", sep='\t')
old_pd_data = pd.read_csv('/Users/vermashresth/Documents/armman-pipeline-latest/may_data/beneficiary/AIRegistration-20200501-20200731.csv')
class_based_features = ['ChannelType', 'education', 'phone_owner', 'income_bracket']

dummy = pd.get_dummies(old_pd_data[class_based_features], columns = class_based_features)
old_pd_data = pd.concat([old_pd_data.drop(columns = class_based_features), dummy], axis=1)
old_pd_data['phone_owner_family'] = old_pd_data['phone_owner_family'] + old_pd_data['phone_owner_Sister']
old_pd_data['phone_owner_neighbor'] = old_pd_data['phone_owner_Nabair ']

dummy = pd.get_dummies(pilot_pd_data[class_based_features], columns = class_based_features)
pilot_pd_data = pd.concat([pilot_pd_data.drop(columns = class_based_features), dummy], axis=1)

g1 = pilot_pd_data
g2 = old_pd_data

numeric_features = ['enroll_gest_age', 'stage', 'age', 'g', 'p', 's', 'l', 'a']
date_based_features = ['lmp', 'registration_date']
class_based_features = dummy.columns

feat_sig_df = pd.DataFrame(columns = ['Feature', 'RMAB Group Mean','Round Robin Group Mean', 'Control Group Mean', 'Joint p-value'])

print('NUMERIC FEATURES STATS')
for f in numeric_features:
    print('-'*60)
    print('Feature {}'.format(f))
    print('G1: Mean - {}, Std - {}'.format(g1[f].mean(), g1[f].std()))
    print('G2: Mean - {}, Std - {}'.format(g2[f].mean(), g2[f].std()))
    print('t test p-val: ', stats.ttest_ind(g1[f], g2[f])[1])

    
print('\n\nCLASS FEATURES STATS')
for f in class_based_features:
    if f not in g1.columns or f not in g2.columns:
        continue
    print('-'*60)
    print('Feature {}'.format(f))
    print('G1: Mean - {}, Std - {}'.format(g1[f].mean(), g1[f].std()))
    print('G2: Mean - {}, Std - {}'.format(g2[f].mean(), g2[f].std()))
    print('t test p-val: ', stats.ttest_ind(g1[f], g2[f])[1])


/Users/vermashresth/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (25,48,49,51,53,54,55,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


NUMERIC FEATURES STATS
------------------------------------------------------------
Feature enroll_gest_age
G1: Mean - 20.382156671058784, Std - 9.599590137459662
G2: Mean - 21.38284810364958, Std - 9.456940473914617
t test p-val:  9.317516449260645e-32
------------------------------------------------------------
Feature stage
G1: Mean - 1.5758490245250139, Std - 1.0228702744679867
G2: Mean - 2.1519716771496364, Std - 1.2006557260956552
t test p-val:  0.0
------------------------------------------------------------
Feature age
G1: Mean - 24.804012411272154, Std - 3.9426912509449594
G2: Mean - 24.265074761779218, Std - 3.846158373072656
t test p-val:  7.802425714229333e-54
------------------------------------------------------------
Feature g
G1: Mean - 1.6883580567008118, Std - 1.1593852505967308
G2: Mean - 1.5960980754020564, Std - 0.7945804128058899
t test p-val:  1.1037453200744399e-25
------------------------------------------------------------
Feature p
G1: Mean - 0.68096229863561

In [22]:
pilot_pd_data.columns
dummy = pd.get_dummies(pilot_pd_data[class_based_features], columns = class_based_features)
pilot_pd_data = pd.concat([pilot_pd_data.drop(columns = class_based_features), dummy], axis=1)
pilot_pd_data.columns

Index(['user_id', 'phone_no', 'lmp', 'enroll_gest_age', 'project_id',
       'manager_id', 'call_slots', 'enroll_delivery_status', 'delivery',
       'language', 'registration_date', 'delivery_date', 'entry_date',
       'deleted', 'phone_type', 'phone_code', 'stage', 'channel_id',
       'updated_time', 'ngo_hosp_id', 'org_type', 'unique_sub_id', 'moved',
       'delete_reason', 'delete_other_reason', 'entry_madeby',
       'entry_updatedby', 'force_delivery_updated', 'completed',
       'dnd_optout_status', 'updated_status_on', 'age', 'enroll_phone_owner',
       'alternate_no', 'alternate_no_owner', 'birth_place', 'name_of_sakhi',
       'name_of_project_officer', 'data_entry_officer', 'g', 'p', 's', 'l',
       'a', 'ppc_bloodpressure', 'ppc_diabetes', 'ppc_cesarean', 'ppc_thyroid',
       'ppc_fibroid', 'ppc_spontaneousAbortion', 'ppc_heightLess140',
       'ppc_pretermDelivery', 'ppc_anaemia', 'ppc_otherComplications',
       'name_of_medication_any', 'planned_place_of_delivery',

In [21]:
pilot_pd_data['phone_owner'].unique()

array(['woman', 'family', 'husband', 'neighbor '], dtype=object)

In [147]:
import scipy.stats as stats

pilot_pd_data = pd.read_csv("feb16-mar15_data/beneficiary/ai_registration-20210216-20210315.csv", sep='\t')
class_based_features = ['ChannelType', 'education', 'phone_owner', 'income_bracket']
dummy = pd.get_dummies(pilot_pd_data[class_based_features], columns = class_based_features)
pilot_pd_data = pd.concat([pilot_pd_data.drop(columns = class_based_features), dummy], axis=1)

g1 = pilot_pd_data[pilot_pd_data['user_id'].isin(rmab_list)]
g2 = pilot_pd_data[pilot_pd_data['user_id'].isin(round_robin_list)]
g3 = pilot_pd_data[pilot_pd_data['user_id'].isin(control_list)]

numeric_features = ['enroll_gest_age', 'stage', 'age', 'g', 'p', 's', 'l', 'a']
date_based_features = ['lmp', 'registration_date']
class_based_features = dummy.columns

feat_sig_df = pd.DataFrame(columns = ['Feature', 'RMAB Group Mean','Round Robin Group Mean', 'Control Group Mean', 'Joint p-value'])

print('NUMERIC FEATURES STATS')
for f in numeric_features:
    print('-'*60)
    print('Feature {}'.format(f))
    print('G1: Mean - {}, Std - {}'.format(g1[f].mean(), g1[f].std()))
    print('G2: Mean - {}, Std - {}'.format(g2[f].mean(), g2[f].std()))
    print('G3: Mean - {}, Std - {}'.format(g3[f].mean(), g3[f].std()))
    p, f_val = stats.f_oneway(g1[f], g2[f], g3[f])
    print(f'p-value: {p}, f-value: {f_val}')
    if np.isnan(p):
        continue
    feat_sig_df.loc[len(feat_sig_df)] = [f, np.round(g1[f].mean(), 3), np.round(g2[f].mean(), 3), np.round(g3[f].mean(), 3), np.round(p, 3)]

# print('\n\nDATE-BASED FEATURES STATS')
# for f in date_based_features:
#     print('-'*60)
#     print('Feature {}'.format(f))
#     g1_date = pd.to_datetime(g1[f], format='%Y-%m-%d') - pd.to_datetime('2018-01-01', format='%Y-%m-%d')
#     g2_date = pd.to_datetime(g2[f], format='%Y-%m-%d') - pd.to_datetime('2018-01-01', format='%Y-%m-%d')
#     g3_date = pd.to_datetime(g3[f], format='%Y-%m-%d') - pd.to_datetime('2018-01-01', format='%Y-%m-%d')
#     print('G1: Mean - {}, Std - {}'.format(g1_date.mean(), g1_date.std()))
#     print('G2: Mean - {}, Std - {}'.format(g2_date.mean(), g2_date.std()))
#     print('G3: Mean - {}, Std - {}'.format(g3_date.mean(), g3_date.std()))
    
print('\n\nCLASS FEATURES STATS')
for f in class_based_features:
    print('-'*60)
    print('Feature {}'.format(f))
    print('G1: Mean - {}, Std - {}'.format(g1[f].mean(), g1[f].std()))
    print('G2: Mean - {}, Std - {}'.format(g2[f].mean(), g2[f].std()))
    print('G3: Mean - {}, Std - {}'.format(g3[f].mean(), g3[f].std()))
    p, f_val = stats.f_oneway(g1[f], g2[f], g3[f])
    print(f'p-value: {p}, f-value: {f_val}')
    if np.isnan(p):
        continue
    feat_sig_df.loc[len(feat_sig_df)] = [f, np.round(g1[f].mean(), 3), np.round(g2[f].mean(), 3), np.round(g3[f].mean(), 3), np.round(p, 3)]



NUMERIC FEATURES STATS
------------------------------------------------------------
Feature enroll_gest_age
G1: Mean - 20.32303077725613, Std - 9.582281515959409
G2: Mean - 20.45409494001043, Std - 9.55710795336926
G3: Mean - 20.324768488326594, Std - 9.616568438330738
p-value: 0.4716117467294041, f-value: 0.6240017683847932
------------------------------------------------------------
Feature stage
G1: Mean - 1.572509128847157, Std - 1.0225502892701166
G2: Mean - 1.572378716744914, Std - 1.019877428412327
G3: Mean - 1.5759749576105386, Std - 1.0231149642154935
p-value: 0.03054908396705614, f-value: 0.9699128630836021
------------------------------------------------------------
Feature age
G1: Mean - 24.798643714136674, Std - 3.965468297061967
G2: Mean - 24.733176838810643, Std - 3.93112875516829
G3: Mean - 24.87844006782314, Std - 3.9353089030842074
p-value: 2.6087728066283824, f-value: 0.07364662576726319
------------------------------------------------------------
Feature g
G1: Mean 

/Users/vermashresth/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


In [148]:
feat_sig_df

,Feature,RMAB Group Mean,Round Robin Group Mean,Control Group Mean,Joint p-value
0,enroll_gest_age,20.323,20.454,20.325,0.472
1,stage,1.573,1.572,1.576,0.031
2,age,24.799,24.733,24.878,2.609
3,g,1.675,1.686,1.705,1.373
4,p,0.678,0.673,0.696,1.570
5,s,0.017,0.019,0.018,0.242
6,l,0.645,0.637,0.662,1.715
7,a,0.036,0.033,0.031,0.901
8,ChannelType_Community,0.707,0.727,0.723,4.356
9,ChannelType_Hospital,0.293,0.273,0.277,4.356
